In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [2]:
import cv2
import numpy as np

In [3]:
!unzip -qq /content/MyDrive/MyDrive/archive.zip

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_dir = '/content/train'
val_dir = '/content/test'

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


In [6]:
train_generator = train_datagen.flow_from_directory(train_dir, (48,48), 
                                                    color_mode='grayscale',
                                                    class_mode='categorical',
                                                    batch_size=64)

val_generator = val_datagen.flow_from_directory(val_dir, (48,48),
                                                color_mode = 'grayscale',
                                                class_mode = 'categorical',                                               
                                                batch_size=64)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, (3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [8]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),
                      metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=val_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
448/448 [==============================] - 311s 693ms/step - loss: 1.8347 - accuracy: 0.2471 - val_loss: 1.7257 - val_accuracy: 0.3207
Epoch 2/50
448/448 [==============================] - 313s 700ms/step - loss: 1.6755 - accuracy: 0.3440 - val_loss: 1.5707 - val_accuracy: 0.4057
Epoch 3/50
448/448 [==============================] - 316s 704ms/step - loss: 1.5555 - accuracy: 0.4017 - val_loss: 1.4693 - val_accuracy: 0.4464
Epoch 4/50
448/448 [==============================] - 312s 697ms/step - loss: 1.4842 - accuracy: 0.4283 - val_loss: 1.4062 - val_accuracy: 0.4670
Epoch 5/50
448/448 [==============================] - 312s 696ms/step - loss: 1.4210 - accuracy: 0.4630 - val_loss: 1.3559 - val_accuracy: 0.4809
Epoch 6/50
448/448 [==============================] - 313s 699ms/step - loss: 1.3758 - accuracy: 0.4790 - val_loss: 1.3128 - val_accuracy: 0.5031
Epoch 7/50
448/448 [==============================] - 312s 697ms/step - loss: 1.3183 - accuracy: 0.5004 - val_loss: 1.2721 -

In [9]:
emotion_model.save_weights('model.h5')

In [15]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: 'anrgy', 1:'disgusted', 2:"Fearful", 3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprised"}

cap= cv2.VideoCapture(0)

while True:
  ret, frame = cap.read()
  if not ret:
    break
  bounding_box = cv2.CascadeClassifier("/content/MyDrive/MyDrive/haarcascade_frontalface_default.xml")
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  faces = bounding_box.detectMultiscale(frame, 1.3, 5 )

  for (x,y,w,h) in faces:
    cv2.rectangle(frame, (x,y-50), (x+w, y+h+10), (255,0,0), 2)
    roi_gray = gray[y:y+h, x:x+w]
    cropped_img = np.expand_dims(cv2.resize(roi_gray, (48,48)), -1, 0)

    emotion_predic = emotion_model.predict(cropped_img)
    max_index = int(np.argmax(emotion_predic))
    cv2.putText(frame, emotion_dict[max_index], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX,1,
                (255,255,255),2 , cv2.LINE_AA)
    cv2.imshow('Video', cv2.reszie(frame, (1200,860), interpolation=cv2.INTER_CUBIC))
    
    if cv2.waitKey(1) or 0xff == ord('q'):
      cap.release()
      cv2.destroyAllWindodws()
      break